# Create Hugging Face Dataset

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/06/notebooks/create-dataset.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/06/notebooks/create-dataset.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install dependencies

In [1]:
!uv pip install datasets

Using Python 3.13.1 environment at: /Users/simon/Dev/CS-394/.venv
Audited 1 package in 11ms


## Configuration

In [2]:
TRAIN_FILE = "../code/train.jsonl"
VALIDATION_FILE = "../code/validation.jsonl"
TEST_FILE = "../code/test.jsonl"

DATASET_REPO = "simonguest/test-dataset"

## Create dataset functions

In [3]:
from datasets import Dataset, DatasetDict
import json


def load_jsonl(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:  # Skip empty lines
                continue
            try:
                # Try parsing the line
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Warning: Error parsing line {line_num}: {e}")
                print(f"Problematic line: {line[:200]}...")
    return data


def create_hf_dataset(train_file, val_file, test_file):
    # Load the data
    train_data = load_jsonl(train_file)
    val_data = load_jsonl(val_file)
    test_data = load_jsonl(test_file)

    # Create datasets
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    test_dataset = Dataset.from_list(test_data)

    # Combine into DatasetDict
    dataset_dict = DatasetDict(
        {
            "train": train_dataset,
            "validation": val_dataset,
            "test": test_dataset,
        }
    )

    return dataset_dict


# Create and validate the dataset is ready to upload
dataset = create_hf_dataset(TRAIN_FILE, VALIDATION_FILE, TEST_FILE)
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"\nSample entry: {dataset['train'][0]}")

Train samples: 10
Validation samples: 3
Test samples: 1

Sample entry: {'messages': [{'content': 'import keyword\nprint(keyword.kwlist)', 'role': 'user'}, {'content': "This code first imports Python's built‑in `keyword` module, which contains information about the language's reserved words. Then it prints the list of all keywords that Python recognizes, so you can see exactly which words are reserved and cannot be used as variable names or function names.\n\nThink of the keyword list like a list of stop signs at an intersection: each stop sign tells drivers (or in this case, the Python interpreter) that they must stop and follow a specific rule. Just as you can't drive through a stop sign, you can't use these words as your own identifiers—they're reserved for special meanings in the language.", 'role': 'assistant'}]}


## Upload to Hugging Face

In [4]:
import os

def upload_dataset(dataset_dict, repo_name, token):
    """Upload dataset to Hugging Face Hub"""
    
    # Push to hub
    dataset_dict.push_to_hub(
        repo_name,
        token=token,
        private=False  # Set to True if you want it private
    )
    
    print(f"Dataset uploaded successfully to: https://huggingface.co/datasets/{repo_name}")

upload_dataset(dataset, DATASET_REPO, os.environ.get("HF_TOKEN"))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Dataset uploaded successfully to: https://huggingface.co/datasets/simonguest/test-dataset
